In [ ]:
# BLOK1:APİDEN VERİ ÇEKME
import pandas as pd
from openaq import OpenAQ
from datetime import datetime, timedelta
import time
from tqdm import tqdm

API_KEY = "beb87bdeb247d901be44e59bd044aa34556b5c4752592b5a2cd9da243a25a466"
BBOX_TR = "26.0433512713,35.8215347357,44.7939896991,42.1414848903" # Türkiye koordinatları
COUNTRY_CODE = 'TR'
RAW_DATA_FILE = 'openaq_raw_data_tr.csv' 

client = OpenAQ(api_key=API_KEY)
# Verilen koordinattaki sensörler çekiliyor
def get_sensor_ids(bbox, client, country_code):
    all_sensor_ids, sensor_country_map = [], {}
    page = 1
    print(f"{country_code} bbox {bbox} içindeki sensörler çekiliyor...")
    while True:
        try:
            resp = client.locations.list(bbox=bbox, limit=1000, page=page)
            if not resp.results: break
            for loc in resp.results:
                if hasattr(loc, 'sensors') and loc.sensors:
                    for sensor in loc.sensors:
                        sensor_id = getattr(sensor, 'id', None)
                        if sensor_id:
                            all_sensor_ids.append(sensor_id)
                            sensor_country_map[sensor_id] = getattr(loc.country, 'code', country_code)
            print(f"Sayfa {page} çekildi, {len(resp.results)} konum eklendi.")
            page += 1; time.sleep(0.2)
        except Exception as e:
            print(f"Hata (Sayfa {page}): {e}. 5 saniye bekleniyor."); time.sleep(5)
            continue
    return list(set(all_sensor_ids)), sensor_country_map
# Sensörlerden günlük verilerl çekiliyor
def get_daily_by_sid(sids, start, end, sensor_map, sleep_time=0.3):
    all_data = []
    print(f"\nToplam {len(sids)} sensör için günlük veri çekiliyor...")
    for sid in tqdm(sids, desc="Sensör Verileri Çekiliyor"):
        page = 1
        while True:
            try:
                resp = client.measurements.list(sensors_id=sid, data="days", datetime_from=start, datetime_to=end, limit=1000, page=page)
                if not resp.results: break
                for res in resp.results:
                    all_data.append({
                        "from_date": res.period.datetime_from.utc,
                        "name": res.parameter.name,
                        "value": res.value,
                        "unit": res.parameter.units,
                        "country": sensor_map.get(sid, 'Unknown')
                    })
                page += 1; time.sleep(sleep_time)
            except Exception as e:
                print(f"\nSensör {sid} için hata (Sayfa {page}): {e}. Bekleniyor...")
                try:
                    wait_time = int(str(e).split('resets in ')[1].split(' ')[0]) + 1
                    print(f"{wait_time} saniye bekleniyor..."); time.sleep(wait_time)
                except:
                    time.sleep(10)
                continue
    return pd.DataFrame(all_data)
#7 yıllık veri alınıyor sadece
bitis = datetime.now()
baslangic = bitis - timedelta(days=365 * 7)
baslangic_str = baslangic.strftime("%Y-%m-%d")
bitis_str = bitis.strftime("%Y-%m-%d")
print(f"Veri çekme aralığı: {baslangic_str} - {bitis_str}")

sensor_ids, sensor_country_map = get_sensor_ids(BBOX_TR, client, COUNTRY_CODE)
if sensor_ids:
    raw_df = get_daily_by_sid(sensor_ids, baslangic_str, bitis_str, sensor_country_map)
    if not raw_df.empty:
        raw_df.to_csv(RAW_DATA_FILE, index=False)
        print(f"\nİşlenmemiş ham veri başarıyla '{RAW_DATA_FILE}' dosyasına kaydedildi")
        print(f"Toplam {len(raw_df)} satır ham veri çekildi")
    else:
        print("API'den veri çekilemedi")
else:
    print("Belirtilen alanda sensör bulunamadı")

Veri çekme aralığı: 2018-07-12 - 2025-07-10
TR bbox 26.0433512713,35.8215347357,44.7939896991,42.1414848903 içindeki sensörler çekiliyor...
Sayfa 1 çekildi, 484 konum eklendi.

Toplam 1862 sensör için GÜNLÜK veri çekiliyor...


Sensör Verileri Çekiliyor:  10%|█         | 191/1862 [08:37<1:11:57,  2.58s/it]


Sensör 5079632 için hata (Sayfa 2): The read operation timed out. Bekleniyor...


Sensör Verileri Çekiliyor:  20%|██        | 377/1862 [16:19<26:38,  1.08s/it]  


Sensör 6359137 için hata (Sayfa 1): Rate limit exceeded. Limit resets in 19 seconds. Bekleniyor...
20 saniye bekleniyor...


...


Sensör 15758 için hata (Sayfa 2): Rate limit exceeded. Limit resets in 2 seconds. Bekleniyor...
3 saniye bekleniyor...


Sensör Verileri Çekiliyor: 100%|█████████▉| 1859/1862 [1:08:38<00:09,  3.13s/it]


Sensör 16175 için hata (Sayfa 1): The operation did not complete (read) (_ssl.c:2559). Bekleniyor...


Sensör Verileri Çekiliyor: 100%|██████████| 1862/1862 [1:09:01<00:00,  2.22s/it]



İşlenmemiş ham veri başarıyla 'openaq_raw_data_tr.csv' dosyasına kaydedildi.
Toplam 663555 satır ham veri çekildi.


In [ ]:
# BLOK2: VERİ ÖN İŞLEME 
import pandas as pd
from tqdm import tqdm

RAW_DATA_FILE = 'openaq_raw_data_tr.csv'
PROCESSED_FILE_PATH = 'tr_air_quality_processed.csv'
COUNTRY_CODE = 'TR'
# Kirleticilerin birimleri eşitleniyor
def process_data(file_path, country_code):
    print(f"'{file_path}' dosyasından ham veri okunuyor...")
    df = pd.read_csv(file_path)
    
    print("\nBirim dönüşümü başlıyor...")
    MOLAR_MASSES = {'co': 28.01, 'no2': 46.01, 'o3': 48.00, 'so2': 64.07} # internetten alınan değerler
    MOLAR_VOLUME = 24.45
    converted_degers = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Birimler Dönüştürülüyor"):
        param, unit, deger = str(row['name']).lower(), str(row['unit']).lower(), row['deger']
        yeni_degeri = None
        if pd.notna(deger) and deger >= 0:
            if unit == 'µg/m³': yeni_degeri = deger
            else:
                molar_mass = MOLAR_MASSES.get(param)
                if molar_mass is not None:
                    if unit == 'ppm': yeni_degeri = (deger * molar_mass * 1000) / MOLAR_VOLUME
                    elif unit == 'ppb': yeni_degeri = (deger * molar_mass) / MOLAR_VOLUME
                    else: yeni_degeri = deger
                else: yeni_degeri = deger
        converted_degers.append(yeni_degeri)
    df['deger_converted'] = converted_degers
    
    # Pivotlama
    print("\nVeri işleme ve pivotlama başlıyor...")
    df['from_date'] = pd.to_datetime(df['from_date'], errors='coerce')
    df.dropna(subset=['from_date', 'deger_converted'], inplace=True)
    df_filtered = df[df['country'] == country_code].copy()
    
    df_pivot = df_filtered.pivot_table(index='from_date', columns='name', degers='deger_converted', aggfunc='mean').reset_index()
    df_pivot.columns.name = None
    df_pivot.sort_degers(by='from_date', ascending=True, inplace=True)
    
    return df_pivot

try:
    processed_df = process_data(RAW_DATA_FILE, COUNTRY_CODE)
    processed_df.to_csv(PROCESSED_FILE_PATH, index=False)
    print(f"\nİşlenmiş (logaritmasız) veri '{PROCESSED_FILE_PATH}' dosyasına kaydedildi.")
    print("İşlenmiş verinin ilk 5 satırı:")
    print(processed_df.head())
except FileNotFoundError:
    print(f"Hata: '{RAW_DATA_FILE}' bulunamadı.")

'openaq_raw_data_tr.csv' dosyasından ham veri okunuyor...

Birim dönüşümü başlıyor...


Birimler Dönüştürülüyor: 100%|██████████| 663555/663555 [00:21<00:00, 30668.45it/s]



Veri işleme ve pivotlama başlıyor...

İşlenmiş (logaritmasız) veri 'tr_air_quality_processed.csv' dosyasına kaydedildi.
İşlenmiş verinin ilk 5 satırı:
                  from_date           co        no2         o3  pm1  \
0 2016-12-11 21:00:00+00:00  4321.000000  53.000000  53.500000  NaN   
1 2016-12-13 21:00:00+00:00          NaN   6.500000  73.000000  NaN   
2 2016-12-16 21:00:00+00:00   280.000000   0.000000  37.000000  NaN   
3 2016-12-21 21:00:00+00:00    84.000000   3.000000  90.000000  NaN   
4 2016-12-22 21:00:00+00:00  1526.944444  59.106897  26.691837  NaN   

         pm10  pm25  pressure  relativehumidity         so2  temperature  \
0  304.500000   NaN       NaN               NaN  198.500000          NaN   
1         NaN   NaN       NaN               NaN    5.000000          NaN   
2   10.000000   NaN       NaN               NaN   16.000000          NaN   
3   15.500000   NaN       NaN               NaN    8.500000          NaN   
4   70.526804   NaN       NaN            

In [ ]:
# BLOK3: FEATURE ENGINEERING
import pandas as pd
import numpy as np

PROCESSED_FILE_PATH = 'tr_air_quality_processed.csv'
ENGINEERED_FILE_PATH = 'tr_air_quality_engineered.csv'

print(f"'{PROCESSED_FILE_PATH}' dosyasından veri okunuyor...")
try:
    df = pd.read_csv(PROCESSED_FILE_PATH, parse_dates=['from_date'])
    df.set_index('from_date', inplace=True)
    print(f"Veri başarıyla yüklendi. Boyut: {df.shape}")
except FileNotFoundError:
    print(f"Hata: '{PROCESSED_FILE_PATH}' bulunamadı")
    exit()

df.sort_index(inplace=True)

# Eksik veri interpolate yöntemi ile dolduruldu
print("\nEksik veri yönetimi başlıyor...")
pollutant_cols = [col for col in df.columns if df[col].dtype in ['float64', 'int64']]
# Ülke sütunu çıkarıldı
if 'country' in pollutant_cols: pollutant_cols.remove('country')

df[pollutant_cols] = df[pollutant_cols].interpolate(method='time', limit_direction='both')
df.fillna(method='ffill', inplace=True); df.fillna(method='bfill', inplace=True)
print("Eksik veri yönetimi tamamlandı")


# Aşırı değerler kırpıldı
print("\nAşırı aykırı değerler %99.9 persentil ile kırpılıyor...")
for col in pollutant_cols:
    ust_limit = df[col].quantile(0.999)
    df[col] = df[col].clip(upper=ust_limit)
    print(f" '{col}' sütunu, üst sınır '{ust_limit:.2f}' ile kırpıldı.")

# Logaritmik dönüşüm yapıldı her bir col için hatayı minizimize etmek lazım çünkü
print("\nLogaritmik dönüşüm uygulanıyor...")
for col in pollutant_cols:
    df[col] = np.log1p(df[col])

# Türetilmiş bazı yararlı öznitelikler ekleniyor
print("\nÖznitelik mühendisliği başlıyor...")
df['year'] = df.index.year; df['month'] = df.index.month; df['day_of_week'] = df.index.dayofweek
for pollutant in pollutant_cols:
    for lag in [1, 3, 7]: df[f'{pollutant}_lag_{lag}d'] = df[pollutant].shift(lag)
    for window in [3, 7]: df[f'{pollutant}_roll_mean_{window}d'] = df[pollutant].rolling(window, closed='left').mean()
df.fillna(method='bfill', inplace=True)
print("Öznitelik mühendisliği tamamlandı")

df.to_csv(ENGINEERED_FILE_PATH, index=True)
print(f"\nTemizlenmiş veri '{ENGINEERED_FILE_PATH}' dosyasına kaydedildi")

'tr_air_quality_processed.csv' dosyasından veri okunuyor...
Veri başarıyla yüklendi. Boyut: (1624, 11)

Eksik veri yönetimi başlıyor...
Eksik veri yönetimi tamamlandı.

Aşırı aykırı değerler %99.9 persentil ile kırpılıyor...
  'co' sütunu, üst sınır '3719868.69' ile kırpıldı.
  'no2' sütunu, üst sınır '187.53' ile kırpıldı.
  'o3' sütunu, üst sınır '598.78' ile kırpıldı.
  'pm1' sütunu, üst sınır '48.47' ile kırpıldı.
  'pm10' sütunu, üst sınır '410914.25' ile kırpıldı.
  'pm25' sütunu, üst sınır '14705.52' ile kırpıldı.
  'pressure' sütunu, üst sınır '1032.75' ile kırpıldı.
  'relativehumidity' sütunu, üst sınır '81.70' ile kırpıldı.
  'so2' sütunu, üst sınır '238.17' ile kırpıldı.
  'temperature' sütunu, üst sınır '30.61' ile kırpıldı.
  'um003' sütunu, üst sınır '13209.32' ile kırpıldı.

Logaritmik dönüşüm uygulanıyor...

Öznitelik mühendisliği başlıyor...
Öznitelik mühendisliği tamamlandı.

Temizlenmiş ve öznitelik mühendisliği yapılmış veri 'tr_air_quality_engineered.csv' dosyasın

C:\Users\melis\AppData\Local\Temp\ipykernel_32320\1264478784.py:28: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True); df.fillna(method='bfill', inplace=True)
C:\Users\melis\AppData\Local\Temp\ipykernel_32320\1264478784.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [ ]:
# BLOK4: TRAİNİNG
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, TimeSeriesSplit
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, f_regression
import joblib
import warnings
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
warnings.filterwarnings('ignore')

COUNTRY_CODE = 'tr' 
ENGINEERED_FILE_PATH = f'{COUNTRY_CODE}_air_quality_engineered.csv'
TARGET_POLLUTANTS = ['co', 'no2', 'o3', 'pm10', 'pm25', 'so2']
FORECAST_HORIZON = 1

# Veri yükleme
df_engineered = pd.read_csv(ENGINEERED_FILE_PATH, index_col='from_date', parse_dates=True)
print(f"'{ENGINEERED_FILE_PATH}' yüklendi. Boyut: {df_engineered.shape}")

# Model tanımları
models = {
    "xgboost": {"estimator": xgb.XGBRegressor(random_state=42), "params": {'n_estimators': [100, 300], 'max_depth': [5, 7]}},
    "randomforest": {"estimator": RandomForestRegressor(random_state=42), "params": {'n_estimators': [100, 200], 'max_depth': [10, 20]}},
    "mlp": {"estimator": MLPRegressor(random_state=42, max_iter=500, early_stopping=True), "params": {'hidden_layer_sizes': [(64, 32)], 'alpha': [0.001, 0.05]}},
    "knn": {"estimator": KNeighborsRegressor(), "params": {'n_neighbors': [7, 15], 'weights': ['uniform', 'distance']}}
}

available_targets = [p for p in TARGET_POLLUTANTS if p in df_engineered.columns]

for model_name, model_info in models.items():
    for target in available_targets:
        print(f"\n>>> {COUNTRY_CODE.upper()}: {model_name.upper()} - {target.upper()} modeli eğitiliyor...")
        
        # 1. Veri hazırlama
        features_to_use = [col for col in df_engineered.columns if col not in TARGET_POLLUTANTS + ['country']]
        X = df_engineered[features_to_use].copy()
        y = df_engineered[[target]].shift(-FORECAST_HORIZON)
        X, y = X.iloc[:-FORECAST_HORIZON], y.iloc[:-FORECAST_HORIZON]
        X.dropna(axis=1, how='all', inplace=True); X.fillna(0, inplace=True)
        
        X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
        
        # Test için nesne oluşturma ve kaydetme
        x_scaler = RobustScaler().fit(X_train_full)
        y_scaler = RobustScaler().fit(y_train_full)
        k_val = min(20 if model_name == 'knn' else 35, X_train_full.shape[1])
        X_train_s = pd.DataFrame(x_scaler.transform(X_train_full), columns=X_train_full.columns)
        selector = SelectKBest(score_func=f_regression, k=k_val).fit(X_train_s, y_train_full.values.ravel())
        
        joblib.dump(x_scaler, f'x_scaler_{model_name}_{target}_{COUNTRY_CODE}.pkl')
        joblib.dump(y_scaler, f'y_scaler_{model_name}_{target}_{COUNTRY_CODE}.pkl')
        joblib.dump(selector, f'selector_{model_name}_{target}_{COUNTRY_CODE}.pkl')
        X_test.to_csv(f'X_test_{target}_{COUNTRY_CODE}.csv')
        y_test.to_csv(f'y_test_{target}_{COUNTRY_CODE}.csv')

        # Hiperparametre optimizasyonu
        X_train_hp_f = selector.transform(X_train_s)
        y_train_hp_s = y_scaler.transform(y_train_full)
        
        tscv = TimeSeriesSplit(n_splits=3)
        random_search = RandomizedSearchCV(
            estimator=model_info["estimator"], param_distributions=model_info["params"],
            n_iter=4, cv=tscv, scoring='neg_root_mean_squared_error', random_state=42, n_jobs=-1
        )
        random_search.fit(X_train_hp_f, y_train_hp_s.ravel())
        best_params = random_search.best_params_
        
        # Final modelinin tüm eğitim verisiyle eğitimi ve kaydı
        final_model = model_info["estimator"].set_params(**best_params)
        final_model.fit(X_train_hp_f, y_train_hp_s.ravel())
        joblib.dump(final_model, f'model_{model_name}_{target}_{COUNTRY_CODE}.pkl')
        
        print(f"-> {target.upper()} için {model_name.upper()} modeli eğitildi ve kaydedildi.")

'tr_air_quality_engineered.csv' yüklendi. Boyut: (1624, 69)

>>> TR: XGBOOST - CO modeli eğitiliyor...
-> CO için XGBOOST modeli eğitildi ve kaydedildi.

>>> TR: XGBOOST - NO2 modeli eğitiliyor...
-> NO2 için XGBOOST modeli eğitildi ve kaydedildi.

>>> TR: XGBOOST - O3 modeli eğitiliyor...
-> O3 için XGBOOST modeli eğitildi ve kaydedildi.

>>> TR: XGBOOST - PM10 modeli eğitiliyor...
-> PM10 için XGBOOST modeli eğitildi ve kaydedildi.

>>> TR: XGBOOST - PM25 modeli eğitiliyor...
-> PM25 için XGBOOST modeli eğitildi ve kaydedildi.

>>> TR: XGBOOST - SO2 modeli eğitiliyor...
-> SO2 için XGBOOST modeli eğitildi ve kaydedildi.

>>> TR: RANDOMFOREST - CO modeli eğitiliyor...
-> CO için RANDOMFOREST modeli eğitildi ve kaydedildi.

>>> TR: RANDOMFOREST - NO2 modeli eğitiliyor...
-> NO2 için RANDOMFOREST modeli eğitildi ve kaydedildi.

>>> TR: RANDOMFOREST - O3 modeli eğitiliyor...
-> O3 için RANDOMFOREST modeli eğitildi ve kaydedildi.

>>> TR: RANDOMFOREST - PM10 modeli eğitiliyor...
-> PM10 i